In [1]:
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def update_path(source_path, new_path_prefix):
    filename = source_path.split('/')[-1]
    return new_path_prefix + filename

driving_log = pd.read_csv('data/driving_log.csv')

In [2]:
num_in = len(driving_log)
X_train = np.zeros((6*num_in, 160, 320, 3), dtype=np.uint8)
y_train = np.zeros(6*num_in, dtype=float)

high_steering = (driving_log.steering < -0.25) | (driving_log.steering > 0.25)

for i, (img_path_orig, steering_angle) in enumerate(zip(driving_log['center'], driving_log['steering'])):
    img_path = update_path(img_path_orig, 'data/IMG/')
    image = plt.imread(img_path)
    X_train[2*i] = image
    y_train[2*i] = steering_angle
    X_train[2*i+1] = np.fliplr(image)
    y_train[2*i+1] = -steering_angle
    
for i, (img_path_orig, steering_angle) in enumerate(zip(driving_log['left'], driving_log['steering'])):
    img_path = update_path(img_path_orig, 'data/IMG/')
    image = plt.imread(img_path)
    steering_angle += 0.25
    X_train[2*num_in + 2*i] = image
    y_train[2*num_in + 2*i] = steering_angle
    X_train[2*num_in + 2*i+1] = np.fliplr(image)
    y_train[2*num_in + 2*i+1] = -steering_angle
    
for i, (img_path_orig, steering_angle) in enumerate(zip(driving_log['right'], driving_log['steering'])):
    img_path = update_path(img_path_orig, 'data/IMG/')
    image = plt.imread(img_path)
    steering_angle -= 0.25
    X_train[4*num_in + 2*i] = image
    y_train[4*num_in + 2*i] = steering_angle
    X_train[4*num_in + 2*i+1] = np.fliplr(image)
    y_train[4*num_in + 2*i+1] = -steering_angle
    

In [34]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Input, Cropping2D, Dropout, Activation, Concatenate
from keras.callbacks import EarlyStopping
from keras.applications import VGG19, VGG16, InceptionV3, ResNet50

dropout_rate = 0.3
crop_top = 60
crop_bottom = 20
split_left_right = 140
stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3)

def my_loss_fn(y_true, y_pred):
    weighted_squared_difference = (y_true - y_pred)**2 * (np.abs(y_true) + 0.01) # what if true = 0?
    return weighted_squared_difference

pretrained = VGG16(weights='imagenet', include_top=False,
                   input_shape=(160-crop_top-crop_bottom, 320-split_left_right, 3))
for layer in pretrained.layers:
    layer.trainable = False
    
# pretrained_cut = Sequential()  # https://github.com/tensorflow/tensorflow/issues/22479
# for layer in pretrained.layers[:-4]:
#     pretrained_cut.add(layer)


common_dense_layer = Dense(128)

def sidewise_block(input_tensor):
    x = Lambda(lambda x: (x / 255.0) - 0.5)(input_tensor)
    x = pretrained(x)
    x = Flatten()(x)
    x = Dropout(rate=dropout_rate)(x)
    x = common_dense_layer(x)
    x = Activation('relu')(x)
    return x

inp = Input(shape=(160,320,3))
left = Cropping2D(cropping=((crop_top,crop_bottom), (0,split_left_right)))(inp)
right = Cropping2D(cropping=((crop_top,crop_bottom), (split_left_right,0)))(inp)
left = sidewise_block(left)
right = sidewise_block(right)
x = Concatenate(axis=1)([left, right])
x = Dropout(rate=dropout_rate)(x)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dropout(rate=dropout_rate)(x)
out = Dense(1)(x)
model = Model(inputs = inp, outputs=out)

# inp = Input(shape=(160,320,3))
# x = Cropping2D(cropping=((60,20), (0,0)))(inp)
# x = Lambda(lambda x: (x / 255.0) - 0.5)(x)
# x = pretrained(x)
# x = Flatten()(x)
# x = Dropout(rate=0.3)(x)
# x = Dense(400)(x)
# x = Activation('relu')(x)
# x = Dropout(rate=0.3)(x)
# x = Dense(1)(x)
# model = Model(inputs = inp, outputs=x)

model.compile(loss=my_loss_fn, optimizer='adam')
history_object = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=30,
                           batch_size=256, callbacks=[stopper])
model.compile(loss='mse', optimizer='adam')

model.save('model_h.h5')

Train on 38572 samples, validate on 9644 samples
Epoch 1/30
38572/38572 [==============================] - 85s 2ms/step - loss: 0.0234 - val_loss: 0.0117
Epoch 2/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.0070 - val_loss: 0.0124
Epoch 3/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.0059 - val_loss: 0.0125
Epoch 4/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.0053 - val_loss: 0.0115
Epoch 5/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.0050 - val_loss: 0.0105
Epoch 6/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.0047 - val_loss: 0.0108
Epoch 7/30
38572/38572 [==============================] - 80s 2ms/step - loss: 0.0046 - val_loss: 0.0103
Epoch 8/30
38572/38572 [==============================] - 80s 2ms/step - loss: 0.0044 - val_loss: 0.0108
Epoch 9/30
38572/38572 [==============================] - 79s 2ms/step - loss: 0.0044 - val_loss: 0.0108
Epoch 

In [ ]:
Train on 38572 samples, validate on 9644 samples
Epoch 1/30
38572/38572 [==============================] - 73s 2ms/step - loss: 1293.8032 - val_loss: 0.2095
Epoch 2/30
38572/38572 [==============================] - 67s 2ms/step - loss: 0.5559 - val_loss: 0.0405
Epoch 3/30
38572/38572 [==============================] - 67s 2ms/step - loss: 0.0506 - val_loss: 0.0373
Epoch 4/30
38572/38572 [==============================] - 67s 2ms/step - loss: 0.0307 - val_loss: 0.0367
Epoch 5/30
38572/38572 [==============================] - 67s 2ms/step - loss: 0.0253 - val_loss: 0.0366
Epoch 6/30
38572/38572 [==============================] - 67s 2ms/step - loss: 0.0213 - val_loss: 0.0364

In [9]:
model.compile(loss='mse', optimizer='adam')
model.save('model_30_custom_loss.h5')

In [12]:
pretrained.layers.pop()
pretrained.layers.pop()
pretrained.layers.pop()
pretrained.layers.pop()

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
cropping2d_11 (Cropping2D)      (None, 80, 180, 3)   0           input_12[0][0]                   
__________________________________________________________________________________________________
cropping2d_12 (Cropping2D)      (None, 80, 180, 3)   0           input_12[0][0]                   
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 80, 180, 3)   0           cropping2d_11[0][0]              
                                                                 cropping2d_12[0][0]              
__________

In [21]:
pretrained.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 80, 180, 3)        0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 80, 180, 64)       1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 80, 180, 64)       36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 40, 90, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 40, 90, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 40, 90, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 20, 45, 128)       0         
__________

In [25]:
pretrained = VGG16(weights='imagenet', include_top=False,
                   input_shape=(160-crop_top-crop_bottom, 320-split_left_right, 3))
pre = Sequential()
for layer in pretrained.layers[:-8]: # just exclude last layer from copying
    pre.add(layer)

In [27]:
pre.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 80, 180, 64)       1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 80, 180, 64)       36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 40, 90, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 40, 90, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 40, 90, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 20, 45, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 20, 45, 256)       295168    
__________

In [28]:
pre.save('test_p.h5')